In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
import glob
import PIL, cv2
import os
from sklearn.model_selection import train_test_split
import sys
sys.path.append('../scripts/')
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from save_weights_every_epoch import CallbackForSavingModelWeights
from swin_transformer import final_model
%load_ext autoreload
%autoreload 2
import tensorflow.keras.backend as K
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

In [2]:
allowed_gpus = [0]
gpus = tf.config.list_physical_devices('GPU')
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, 'GPU')
for gpu in final_gpu_list:
    tf.config.experimental.set_memory_growth(gpu, True)
strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [3]:
csv_file = pd.read_csv('../../files/train.csv')

In [4]:
csv_file.head()

,image_id,center_id,patient_id,image_num,label
0,006388_0,11,006388,0,CE
1,008e5c_0,11,008e5c,0,CE
2,00c058_0,11,00c058,0,LAA
3,01adc5_0,11,01adc5,0,LAA
4,026c97_0,4,026c97,0,CE


In [5]:
csv_file['fixed_paths'] = csv_file['image_id'].apply(lambda x: '../../files/resized_train/' + x + '.png')

In [6]:
csv_file.head()

,image_id,center_id,patient_id,image_num,label,fixed_paths
0,006388_0,11,006388,0,CE,../../files/resized_train/006388_0.png
1,008e5c_0,11,008e5c,0,CE,../../files/resized_train/008e5c_0.png
2,00c058_0,11,00c058,0,LAA,../../files/resized_train/00c058_0.png
3,01adc5_0,11,01adc5,0,LAA,../../files/resized_train/01adc5_0.png
4,026c97_0,4,026c97,0,CE,../../files/resized_train/026c97_0.png


In [7]:
for x in csv_file['fixed_paths'].values:
    if os.path.exists(x) == False:
        print(x)

In [8]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size=test_size)
    train, val = train_test_split(train, test_size=test_size)
    return train, val, test

In [9]:
def read_imgs(img, label):
    img = tf.io.read_file(img)
    img = tf.image.decode_png(img, channels=3)
    img = img / 255
    return img, label

In [10]:
def get_data(csv_file, repeat=True, shuffle=True, batch=True, batch_size=16):
    imgs, labels = csv_file['fixed_paths'].values.tolist(), [1 if x == 'CE' else 0 for x in csv_file['label'].values.tolist()]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, labels))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_imgs, num_parallel_calls=AUTO)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [11]:
train, val, test = split_datasets(csv_file, test_size=0.05)

In [12]:
train_dataset = get_data(train)
val_dataset = get_data(val)
test_dataset = get_data(test)

In [13]:
input_shape = (256, 256, 3)
patch_size = (2,2)
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
embed_dim = 64
num_heads = 8
window_size = 2
dropout_rate = 0.0
qkv_bias = True
num_mlp = 256
shift_size = 1

In [14]:
test_arr = np.random.random((10, 256, 256, 3))

In [15]:
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [16]:
K.clear_session()
# log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_32/"
# if os.path.exists(log_dir) == False:
#     os.makedirs(log_dir)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../weights_mayo_strip_ai/swin_version_1/'
weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 4
train_dataset = get_data(train, batch_size=batch_size)
val_dataset = get_data(val, shuffle=False, repeat=False, batch_size=batch_size)
# model_config_file
input_shape = (256, 256, 3)
patch_size = (2, 2)
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
embed_dim = 128
num_heads = 16
window_size = 2
dropout_rate = 0.0
qkv_bias = False
num_mlp = 1024
shift_size = 1
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)
wandb.init(
    project = 'mayo_strip_ai',
    # Please add number of epochs
    config = {
        'input_shape' : input_shape,
        'patch_size' : patch_size,
        'num_patch_x' : num_patch_x,
        'num_patch_y' : num_patch_y, 
        'embed_dim' : embed_dim,
        'num_heads' : num_heads,
        'window_size' : window_size,
        'dropout_rate' : dropout_rate,
        'qkv_bias' : qkv_bias,
        'num_mlp' : num_mlp,
        'shift_size' : shift_size,
        'optim' : model.optimizer,
        'batch_size' : batch_size,
    }
)
# model traning loop
with strategy.scope():
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 100,
        verbose = 1,
        validation_data = val_dataset,
        # callbacks = [tensorboard_callback]
        callbacks = [
            WandbMetricsLogger()
        ]
    )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashchks87. Use `wandb login --relogin` to force relogin


Epoch 1/100
170/170 [==============================] - 72s 373ms/step - loss: 0.9248 - prec: 0.7495 - rec: 0.8320 - val_loss: 0.7181 - val_prec: 0.0000e+00 - val_rec: 0.0000e+00
Epoch 2/100
170/170 [==============================] - 61s 360ms/step - loss: 0.7983 - prec: 0.7451 - rec: 0.7691 - val_loss: 0.7603 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 3/100
170/170 [==============================] - 61s 358ms/step - loss: 0.7020 - prec: 0.7245 - rec: 0.8839 - val_loss: 0.6199 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 4/100
170/170 [==============================] - 61s 359ms/step - loss: 0.6186 - prec: 0.7192 - rec: 0.9776 - val_loss: 0.5789 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 5/100
170/170 [==============================] - 61s 359ms/step - loss: 0.5973 - prec: 0.7432 - rec: 0.9821 - val_loss: 0.7722 - val_prec: 0.8889 - val_rec: 0.2963
Epoch 6/100
170/170 [==============================] - 61s 357ms/step - loss: 0.6149 - prec: 0.7377 - rec: 0.9841 - val_loss: 0.5981 - val_pre

In [17]:
K.clear_session()
# log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_32/"
# if os.path.exists(log_dir) == False:
#     os.makedirs(log_dir)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../weights_mayo_strip_ai/swin_version_2/'
weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 4
train_dataset = get_data(train, batch_size=batch_size)
val_dataset = get_data(val, shuffle=False, repeat=False, batch_size=batch_size)
# model_config_file
input_shape = (256, 256, 3)
patch_size = (2, 2)
num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]
embed_dim = 64
num_heads = 8
window_size = 2
dropout_rate = 0.0
qkv_bias = True
num_mlp = 256
shift_size = 1
with strategy.scope():
    model = final_model(input_shape, patch_size, embed_dim, num_heads, window_size, dropout_rate, qkv_bias, num_mlp, shift_size)
wandb.init(
    project = 'mayo_strip_ai',
    # Please add number of epochs
    config = {
        'input_shape' : input_shape,
        'patch_size' : patch_size,
        'num_patch_x' : num_patch_x,
        'num_patch_y' : num_patch_y, 
        'embed_dim' : embed_dim,
        'num_heads' : num_heads,
        'window_size' : window_size,
        'dropout_rate' : dropout_rate,
        'qkv_bias' : qkv_bias,
        'num_mlp' : num_mlp,
        'shift_size' : shift_size,
        'optim' : model.optimizer,
        'batch_size' : batch_size,
    }
)
# model traning loop
with strategy.scope():
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 100,
        verbose = 1,
        validation_data = val_dataset,
        # callbacks = [tensorboard_callback]
        callbacks = [
            WandbMetricsLogger()
        ]
    )

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▆▅▃▄▂▁▁▁▂▄▂▁▆▆▆▂▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁
epoch/prec,▂▁▁▁▂▅▅▇███▇▆██▁▃▄▇████▇██████▅▇████████
epoch/rec,▁▃▇██▆▇▇▇██▇▆▇█▄▅▅██████▇█████▇▇████████
epoch/val_loss,▁▁▁▁▁▁▁▂▂▂▃▃▃▁▄▁▁▁▃▄▅▆▆▃▄▅▆▆██▂▂▂▃▃▃▄▄▄▄
epoch/val_prec,▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████▇▇
epoch/val_rec,▁█████▆▇█▇▇▇▇█▇███▇▆▆▆▆▆▆▇▇▇▇▆▅▆▇▅▅▆▇▆▆▆
epoch/epoch,99
epoch/learning_rate,0.001
epoch/loss,0.00013


Epoch 1/100
170/170 [==============================] - 36s 164ms/step - loss: 0.7550 - prec: 0.7259 - rec: 0.8522 - val_loss: 0.5675 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 2/100
170/170 [==============================] - 25s 150ms/step - loss: 0.6481 - prec: 0.7288 - rec: 0.9678 - val_loss: 0.5898 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 3/100
170/170 [==============================] - 25s 149ms/step - loss: 0.6313 - prec: 0.7317 - rec: 0.9343 - val_loss: 0.5668 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 4/100
170/170 [==============================] - 25s 149ms/step - loss: 0.6213 - prec: 0.7192 - rec: 0.9878 - val_loss: 0.5717 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 5/100
170/170 [==============================] - 25s 149ms/step - loss: 0.5960 - prec: 0.7353 - rec: 1.0000 - val_loss: 0.5760 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 6/100
170/170 [==============================] - 25s 149ms/step - loss: 0.6036 - prec: 0.7339 - rec: 0.9639 - val_loss: 0.6375 - val_prec: 0.750

In [24]:
from incep_model import get_incep_model

In [26]:
K.clear_session()
# log_dir = f"{os.environ['tb_path']}segmentation/res50_baseline_bs_128_is_32/"
# if os.path.exists(log_dir) == False:
#     os.makedirs(log_dir)
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../weights_mayo_strip_ai/incep_v1/'
weights_save = CallbackForSavingModelWeights(weights_path)
batch_size = 4
train_dataset = get_data(train, batch_size=batch_size)
val_dataset = get_data(val, shuffle=False, repeat=False, batch_size=batch_size)
# model_config_file
with strategy.scope():
    model = get_incep_model()
wandb.init(
    project = 'mayo_strip_ai',
    # Please add number of epochs
    config = {
        'model_name' : 'InceptionV3',
        # 'input_shape' : input_shape,
        # 'patch_size' : patch_size,
        # 'num_patch_x' : num_patch_x,
        # 'num_patch_y' : num_patch_y, 
        # 'embed_dim' : embed_dim,
        # 'num_heads' : num_heads,
        # 'window_size' : window_size,
        # 'dropout_rate' : dropout_rate,
        # 'qkv_bias' : qkv_bias,
        # 'num_mlp' : num_mlp,
        # 'shift_size' : shift_size,
        'optim' : model.optimizer,
        'batch_size' : batch_size,
    }
)
# model traning loop
with strategy.scope():
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 100,
        verbose = 1,
        validation_data = val_dataset,
        # callbacks = [tensorboard_callback]
        callbacks = [
            WandbMetricsLogger()
        ]
    )

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▇▇▂▁▁▁▁▁▁▁▁▃▁▁▁▁▁▂▂▁▁▁▁▁▁▁▅▂▁▁▁▁▁▂▁▁▁▁
epoch/prec,▁▁▁▁▇█▇██████▇▇█████▇███████▃▇██▇███████
epoch/rec,▁▅▆▇▇████████▇█████▇▇███████▆██████▇████
epoch/val_loss,▁▁▁▁▂▂▁▂▂▃▃▃▄▂▄▄▅▆▆▇▃▅▆▇█▇██▁▄▅▆▅▄█▅▃▆▇█
epoch/val_prec,▃▃▃▂▃▅▄▄▄▅▄▅█▃▄▅▅▅▅▂▄▂▄▄▃▅▆▅▁▅▅▃▂▇▇▄▆▆▇▇
epoch/val_rec,███▇▅▃▄▅▆▅▃▃▃▅▃▄▄▄▄▇▁▂▄▄▄▄▅▄▅▃▄▄▃▄▄▃▂▅▅▄
epoch/epoch,99
epoch/learning_rate,0.001
epoch/loss,0.00293


Epoch 1/100
170/170 [==============================] - 48s 121ms/step - loss: 0.7008 - prec: 0.7256 - rec: 0.8690 - val_loss: 0.5657 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 2/100
170/170 [==============================] - 17s 100ms/step - loss: 0.6140 - prec: 0.7449 - rec: 0.9504 - val_loss: 0.5665 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 3/100
170/170 [==============================] - 17s 101ms/step - loss: 0.6056 - prec: 0.7307 - rec: 0.9840 - val_loss: 0.5703 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 4/100
170/170 [==============================] - 17s 101ms/step - loss: 0.6332 - prec: 0.7100 - rec: 0.9711 - val_loss: 70.8425 - val_prec: 0.0000e+00 - val_rec: 0.0000e+00
Epoch 5/100
170/170 [==============================] - 16s 95ms/step - loss: 0.6204 - prec: 0.7241 - rec: 0.9758 - val_loss: 0.6193 - val_prec: 0.7500 - val_rec: 1.0000
Epoch 6/100
170/170 [==============================] - 16s 93ms/step - loss: 0.6077 - prec: 0.7282 - rec: 0.9960 - val_loss: 0.5557 - val_prec